#Inteligencia Artificial 2019-1 Proyecto Agricultura

Cristian Andrés Picón Rodríguez - 2160082\
Andrea Juliana Villalba Santos    - 2162114 \

## Import lines

In [ ]:
import os
import sys

import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import KFold
from sklearn.metrics import *


from google.colab import drive
drive.mount("/content/drive/")

##_ Creation of the useful DataSet

### Inicio del tratamiento de datos:

In [ ]:
DATA_DIR   = "/content/drive/My Drive/AI_2019/Project/Project_Data/"
csvList    = []
csvMapName = {}
for index,csvName in enumerate(os.listdir(DATA_DIR)):
  csvList.append(pd.read_csv(DATA_DIR + csvName))
  structure = csvName.split('_')
  #print(csvName)
  csvMapName.update({
      structure[2] : csvList[index]
  }) 

In [ ]:
#Quitar las columnas que nos sirven del dataset cargado y ponerlas en el nuevo.
csvDataMap  = {}
csvDataList = []
for index,i in enumerate(csvMapName):
  Dtemp = pd.DataFrame(csvMapName[i])
  tempCol = Dtemp.columns
  csvDataset = pd.DataFrame({tempCol[1]:Dtemp[tempCol[1]],
                            tempCol[3]:Dtemp[tempCol[3]],
                            tempCol[5]:Dtemp[tempCol[5]],
                            tempCol[10]:Dtemp[tempCol[10]],
                            tempCol[11]:Dtemp[tempCol[11]],
                            tempCol[12]:Dtemp[tempCol[12]],
                            tempCol[13]:Dtemp[tempCol[13]],
                            tempCol[14]:Dtemp[tempCol[14]]})
  csvDataMap.update({i:csvDataset})
for i in csvDataMap:
  csvDataMap[i].columns=['DEPARTAMENTO','MUNICIPIO','CULTIVO',
                                'PERIODO','Área Sembrada(ha)','Área Cosechada(ha)',
                                'Producción(t)','Rendimiento(t/ha)']

In [ ]:

csvDataMap['Maiz'].head()

In [ ]:
csvDataMap['Maiz'].info()
csvDataMap['Maiz'].describe()


Para el tratamiento de datos es necesario reemplazar todos los datos alfanúmericos en el Data Frame por valores númericos. \
En los siguientes Dataframes se encuentran los Departamentos, Municipios, Subgrupo de Cultivo y Perido con sus respectivos ID.

In [ ]:
DepId = {}
tempFullMap = []
tempRow = []
newData = pd.DataFrame()
for index,i in enumerate(csvDataMap):
  tempData = csvDataMap[i]        #Cada Dataset 
  tempCol = tempData.columns #Nombre de las columnas del Dataset
  tempColData = tempData[tempCol[0]].drop_duplicates()
  for k in tempColData:
    tempRow.append(k)
final = pd.Series(sorted(tempRow)).drop_duplicates()
final = final.reset_index()
del final['index']
final = final.rename(columns = {0:'Departamento'})
for m,n in enumerate(final['Departamento']):
  DepId.update({m:n})
print(final.shape)
final.head(33)

In [ ]:
for index,i in enumerate(csvDataMap):    #Cada Dataset 
  for j in final['Departamento']:        #Cada Departamento
    cont = csvDataMap[i][csvDataMap[i].DEPARTAMENTO==j]['DEPARTAMENTO'].index
    if(cont.size > 0):
      for p in cont:
        csvDataMap[i].loc[p,'DEPARTAMENTO'] = list(DepId.keys())[list(DepId.values()).index(j)]        

In [ ]:
MunId = {}
tempFullMap = []
tempRow = []
newData = pd.DataFrame()
for index,i in enumerate(csvDataMap):
  tempData = csvDataMap[i]        #Cada Dataset 
  tempCol = csvDataMap[i].columns #Nombre de las columnas del Dataset
  tempColData = tempData[tempCol[1]].drop_duplicates()
  for k in tempColData:
    tempRow.append(k)
final = pd.Series(sorted(tempRow)).drop_duplicates()
final = final.reset_index()
del final['index']
final = final.rename(columns = {0:'Municipio'})
for m,n in enumerate(final['Municipio']):
  MunId.update({m:n})
print(final.shape)
final.head(5)

In [ ]:
for index,i in enumerate(csvDataMap):    #Cada Dataset 
  for j in final['Municipio']:        #Cada Departamento
    cont = csvDataMap[i][csvDataMap[i].MUNICIPIO==j]['MUNICIPIO'].index
    if(cont.size > 0):
      for p in cont:
        csvDataMap[i].loc[p,'MUNICIPIO'] = list(MunId.keys())[list(MunId.values()).index(j)]        

In [ ]:
GrupId = {}
tempFullMap = []
tempRow = []
newData = pd.DataFrame()
for index,i in enumerate(csvDataMap):
  tempData = csvDataMap[i]        #Cada Dataset 
  tempCol = csvDataMap[i].columns #Nombre de las columnas del Dataset
  tempColData = tempData[tempCol[2]].drop_duplicates()
  for k in tempColData:
    tempRow.append(k)
final = pd.Series(sorted(tempRow)).drop_duplicates()
final = final.reset_index()
del final['index']
final = final.rename(columns = {0:'Cultivo'})
for m,n in enumerate(final['Cultivo']):
  GrupId.update({m:n})
final.head()

In [ ]:
for index,i in enumerate(csvDataMap):    #Cada Dataset 
  for j in final['Cultivo']:        #Cada Departamento
    cont = csvDataMap[i][csvDataMap[i].CULTIVO==j]['CULTIVO'].index
    if(cont.size > 0):
      for p in cont:
        csvDataMap[i].loc[p,'CULTIVO'] = list(GrupId.keys())[list(GrupId.values()).index(j)]     

In [ ]:
PerId = {}
tempFullMap = []
tempRow = []
newData = pd.DataFrame()
for index,i in enumerate(csvDataMap):
  tempData = csvDataMap[i]        #Cada Dataset 
  tempCol = csvDataMap[i].columns  #Nombre de las columnas del Dataset
  tempColData = tempData[tempCol[3]].drop_duplicates()
  for k in tempColData:
    tempRow.append(k)
final = pd.Series(tempRow).drop_duplicates()
final = final.reset_index()
del final['index']
final = final.rename(columns = {0:'Periodo'})
for m,n in enumerate(final['Periodo']):
  PerId.update({m:n})
final.head(30)

In [ ]:
for index,i in enumerate(csvDataMap):    #Cada Dataset 
  for j in final['Periodo']:        #Cada Periodo
    cont = csvDataMap[i][csvDataMap[i].PERIODO==j]['PERIODO'].index
    if(cont.size > 0):
      for p in cont:
        csvDataMap[i].loc[p,'PERIODO'] = list(PerId.keys())[list(PerId.values()).index(j)]     

In [ ]:
csvDataMap['Maiz'].head()


**Guardar los Datasets obtenidos luego del tratamiento.**

In [ ]:
DATA_DIR2 = "/content/drive/My Drive/AI_2019/Project/DataT/"
for i in csvDataMap: # csvDataMap es el diccionario que contiene todos los dataset
  csvDataMap[i].to_csv(DATA_DIR2+i+'.csv')
  

### Obtener los Datasets tratados. (Datos finales)

In [ ]:
DATA_DIR   = "/content/drive/My Drive/AI_2019/Project/DataT/"
csvList    = []
csvMapName = {}
for index,csvName in enumerate(os.listdir(DATA_DIR)):
  csvList.append(pd.read_csv(DATA_DIR + csvName))
  structure = csvName.split('.')
  #print(csvName)
  csvMapName.update({
      structure[0] : csvList[index]
  }) 

In [ ]:
for i in csvMapName:
  del csvMapName[i][csvMapName[i].columns[0]]
  #csvMapName[i].to_csv(DATA_DIR+i+'2.csv')

##The variable csvMapName is a Dictionary with the Datasets
\
The Keys to get every Datasets are:

In [ ]:
keys=list(csvDataMap.keys())  ##Listar las keys para acceder a los DataSets
print(keys)

We can access to every Dataset using : 
\
**csvMapName\['key'] = csvMapName\['Maiz']**

In [ ]:
f = csvDataMap[keys[5]]
f.head(2)


In [ ]:
data = f.iloc[:, [1,3,6,7]].values  
data.shape

In [ ]:
from sklearn.datasets import *
import numpy as np  
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.cluster import KMeans 
from sklearn.cluster import DBSCAN




In [ ]:
kmeans = KMeans(n_clusters=5)  
kmeans.fit(data)  

DBS = DBSCAN(eps=20000, min_samples=20)
#AgglomerativeClustering(n_clusters=2, linkage= "ward")
DBS.fit(data) 
plt.figure(figsize=(15, 5))  
plt.subplot(131)
plt.scatter(data[:,1], data[:,2], cmap='rainbow') 
plt.subplot(132)
plt.title("BDScan")
plt.xlabel("periodo")
plt.ylabel("produccion")
plt.grid(alpha=0.2)
plt.scatter(data[:,1], data[:,2], c=DBS.labels_, cmap='rainbow')  
plt.subplot(133)
plt.scatter(data[:,1], data[:,2], c=kmeans.labels_, cmap='rainbow') 
plt.xlabel("periodo")
plt.ylabel("produccion")
plt.title("K-means")
plt.grid(alpha=0.2)

In [ ]:

kmeans = KMeans(n_clusters=3)  
kmeans.fit(data)  

#DBS = DBSCAN(eps=10)
#AgglomerativeClustering(n_clusters=2, linkage= "ward")
DBS.fit(data) 
plt.figure(figsize=(15, 5))  
plt.subplot(131)

plt.scatter(data[:,0], data[:,3], c=kmeans.labels_, cmap='rainbow',alpha=0.5) 
plt.xlabel("municipios")
plt.ylabel("rendimiento")
plt.title("K-means")
plt.grid(alpha=0.5)

In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
plt.rcParams['figure.figsize'] = (10, 9)
plt.style.use('ggplot')

In [ ]:
data = f.iloc[:, [1,3,7]].values  
data.shape

kmeans = KMeans(n_clusters=5).fit(data)
centroids = kmeans.cluster_centers_


# Predicting the clusters
labels = kmeans.predict(data)
# Getting the cluster centers
C = kmeans.cluster_centers_
colores=['red','green','blue','cyan','yellow']
asignar=[]
for row in labels:
    asignar.append(colores[row])

fig = plt.figure()
ax = Axes3D(fig)
ax.scatter(data[:, 0], data[:, 1], data[:, 2], c=asignar,s=30)
ax.scatter(C[:, 0], C[:, 1], C[:, 2], marker='*', c=colores, s=1000)

In [ ]:
f = csvDataMap[keys[1]]


In [ ]:

# library & dataset
import seaborn as sns
 
  
# use the function regplot to make a scatterplot
sns.regplot(x=f["Área Sembrada(ha)"], y=f["Área Cosechada(ha)"])
#sns.plt.show()
 
# Without regression fit:
sns.regplot(x=f["Área Sembrada(ha)"], y=f["Área Cosechada(ha)"], fit_reg=False)
#sns.plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
pd.__version__

f.hist('MUNICIPIO')

## Graphics

In [ ]:
locations = pd.read_csv("/content/drive/My Drive/AI_2019/Project/departamentoscolombia.csv")
del locations["numero_dep"]
locations.head()

In [ ]:
for i in range(len(locations)):  #Asignar un Id a cada Departamento
    locations["departamento"][i] = i  
locations.head()

**EVITAR LA CREACION DE LOS NAN EN LOS DATASETS QUE NO TIENEN EL DEPARTAMENTO**

In [ ]:
prom_prod = []
for i in range(len(csvDataMap)): ##Promedia la producción por Departamento de cada Dataset
  t = csvDataMap[keys[i]]
  prom_prod.append([])
  for j in range(len(locations)):
    prom_prod[i].append(np.mean(t[t.DEPARTAMENTO==j]["Producción(t)"]))
  locations["prom" + keys[i]] = prom_prod[i]

In [ ]:
locations.head()

**Colocar un analisis de los datos con respecto a la produccion**

***Avocado production***


In [ ]:
y = locations["latitud"]
x = locations["longitud"]
n = locations["departamento"]

In [ ]:
#fig, ax = plt.subplots()
plt.figure(figsize=(10,5))
plt.scatter(x, y, alpha=0.5, s=locations["promAguacate"]*0.5, c=locations["departamento"], cmap=plt.get_cmap("jet"))
plt.title('Producción en los departamentos')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.colorbar();

for i, txt in enumerate(n):
    plt.annotate(txt, (x[i], y[i]))

***Cocoa production***

In [ ]:
#fig, ax = plt.subplots()
plt.figure(figsize=(10,5))
plt.scatter(x, y, alpha=0.5, s=locations["promCacao"]*0.5, c=locations["departamento"], cmap=plt.get_cmap("jet"))
plt.title('Producción en los departamentos')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.colorbar();

for i, txt in enumerate(n):
    plt.annotate(txt, (x[i], y[i]))

***Coffee production***

In [ ]:
#fig, ax = plt.subplots()
plt.figure(figsize=(10,5))
plt.scatter(x, y, alpha=0.5, s=locations["promCafe"]*0.5, c=locations["departamento"], cmap=plt.get_cmap("jet"))
plt.title('Producción en los departamentos')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.colorbar();

for i, txt in enumerate(n):
    plt.annotate(txt, (x[i], y[i]))

***Corn production***

In [ ]:
#fig, ax = plt.subplots()
plt.figure(figsize=(10,5))
plt.scatter(x, y, alpha=0.5, s=locations["promMaiz"]*0.5, c=locations["departamento"], cmap=plt.get_cmap("jet"))
plt.title('Producción en los departamentos')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.colorbar();

for i, txt in enumerate(n):
    plt.annotate(txt, (x[i], y[i]))

***Soy production***

In [ ]:
#fig, ax = plt.subplots()
plt.figure(figsize=(10,5))
plt.scatter(x, y, alpha=0.5, s=locations["promSoya"]*0.5, c=locations["departamento"], cmap=plt.get_cmap("jet"))
plt.title('Producción en los departamentos')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.colorbar();

for i, txt in enumerate(n):
    plt.annotate(txt, (x[i], y[i]))

***Tomato production***

In [ ]:
#fig, ax = plt.subplots()
plt.figure(figsize=(10,5))
plt.scatter(x, y, alpha=0.5, s=locations["promTomate"]*0.5, c=locations["departamento"], cmap=plt.get_cmap("jet"))
plt.title('Producción en los departamentos')
plt.xlabel('Longitud')
plt.ylabel('Latitud')
plt.colorbar();

for i, txt in enumerate(n):
    plt.annotate(txt, (x[i], y[i]))

## Data Treatment - Regression


In [ ]:
dataset = csvMapName['Cafe']
dataset.head()

In [ ]:
pd.plotting.scatter_matrix(dataset, figsize=(15,15));

In [ ]:
dataset.describe()

In [ ]:
dataset.head()

In [ ]:
X = dataset.values[:,[0,2,3,4,5]]
y = dataset.values[:,6:7]

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
def show_curve(est):
    means, stds = [], []
    nfolds_range = range(2,11)
    for nfolds in nfolds_range:
      #print(nfolds,)
      f = cross_val_score(est, X, y.ravel(), cv=KFold(nfolds, shuffle=True), scoring=make_scorer(mean_squared_error))
      means.append(np.mean(f))
      stds.append(np.std(f))

    means = np.r_[means]
    stds  = np.r_[stds]

    plt.plot(nfolds_range, means, label="mean", color="black")
    plt.fill_between(nfolds_range, means-stds, means+stds, color="blue", alpha=.5, label="std")
    plt.xlabel("n folds")
    plt.ylabel("performance")
    plt.legend()

In [ ]:
show_curve(DecisionTreeRegressor())

In [ ]:
# Normalizacion de los datos de entrenamiento y test
Xnormalizer = StandardScaler().fit(X_train)
Ynormalizer = StandardScaler().fit(y_train)
XTrain = Xnormalizer.transform(X_train)
YTrain = Ynormalizer.transform(y_train)
XTest = Xnormalizer.transform(X_test)


DTR = DecisionTreeRegressor()
DTR.fit(XTrain,y_train)

s = cross_val_score(DTR, XTrain, YTrain, cv=KFold(10, shuffle=True), scoring=make_scorer(mean_squared_error))
print("MSE %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))

In [ ]:
show_curve(SVR(gamma='auto'))

In [ ]:
from sklearn.svm import SVR
svr = SVR(gamma='auto')
svr.fit(X,y.ravel())
s = cross_val_score(svr, XTrain, YTrain.ravel(), cv=KFold(10, shuffle=True), scoring=make_scorer(mean_squared_error))
print("MSE %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))

In [ ]:
from sklearn.ensemble import RandomForestRegressor


show_curve(RandomForestRegressor(n_estimators = 100))

In [ ]:
RFR = RandomForestRegressor(n_estimators=100)
RFR.fit(X, y.ravel())
s = cross_val_score(RFR, XTrain, YTrain.ravel(), cv=KFold(10, shuffle=True), scoring=make_scorer(mean_squared_error))
print("MSE %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))

## Data treatment - Classification

In [ ]:
for i in csvMapName:
  csvMapName[i]["class"] = ((csvMapName[i]["Área Cosechada(ha)"] > (csvMapName[i]["Área Sembrada(ha)"]-2000)) & (csvMapName[i]["Área Cosechada(ha)"] <= csvMapName[i]["Área Sembrada(ha)"]))*1
  


In [ ]:
datasetC = csvMapName["Cafe"]
datasetC.head()

In [ ]:
from sklearn.model_selection import cross_val_score

X = datasetC.values[:,[0,2,4,5,6]]
y = datasetC.values[:,-1]

In [ ]:
def tpr(est,X,y):
    p = est.predict(X)
    return np.mean (p[y==1] == y[y==1])

def tnr(est,X,y):
    p = est.predict(X)
    return np.mean(p[y==0] == y[y==0])

In [ ]:
from sklearn.naive_bayes import GaussianNB


est = GaussianNB()

s = cross_val_score(est, X, y, cv=KFold(8, shuffle=True), scoring=make_scorer(accuracy_score))
print("accuracy %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))
s = cross_val_score(est, X, y, cv=KFold(8, shuffle=True), scoring=tpr)
print ("tpr %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))
s = cross_val_score(est, X, y, cv=KFold(8, shuffle=True), scoring=tnr)
print ("tnr %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))

In [ ]:
from sklearn.tree import DecisionTreeClassifier

est = DecisionTreeClassifier()

s = cross_val_score(est, X, y, cv=KFold(8, shuffle=True), scoring=make_scorer(accuracy_score))
print("accuracy %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))
s = cross_val_score(est, X, y, cv=KFold(8, shuffle=True), scoring=tpr)
print ("tpr      %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))
s = cross_val_score(est, X, y, cv=KFold(8, shuffle=True), scoring=tnr)
print ("tnr      %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))

In [ ]:
est.fit
est.predict([1,2,10380,7648,9888])

In [ ]:
from sklearn.ensemble import RandomForestClassifier

est = RandomForestClassifier(n_estimators = 100)

s = cross_val_score(est, X, y, cv=KFold(10, shuffle=True), scoring=make_scorer(accuracy_score))
print ("accuracy %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))
s = cross_val_score(est, X, y, cv=KFold(10, shuffle=True), scoring=tpr)
print ("tpr      %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))
s = cross_val_score(est, X, y, cv=KFold(10, shuffle=True), scoring=tnr)
print ("tnr      %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))

In [ ]:
from sklearn.svm import SVC

est = SVC(gamma = 'auto')

s = cross_val_score(est, X, y, cv=KFold(10, shuffle=True), scoring=make_scorer(accuracy_score))
print ("accuracy %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))
s = cross_val_score(est, X, y, cv=KFold(10, shuffle=True), scoring=tpr)
print ("tpr      %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))
s = cross_val_score(est, X, y, cv=KFold(10, shuffle=True), scoring=tnr)
print ("tnr      %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))

In [ ]:
from sklearn.ensemble import RandomForestClassifier
dt =DecisionTreeClassifier()#SVC()#RandomForestClassifier()

X = datasetC.values[:,[0,2,4,5,6]]
y = datasetC.values[:,-1]

Xtr, Xts, ytr, yts = train_test_split(X,y, test_size=.5)

dt.fit(Xtr,ytr)
print (dt.score(Xts,yts))

In [ ]:
from sklearn.decomposition import PCA
from sklearn.naive_bayes import GaussianNB

from sklearn.model_selection import cross_val_score, KFold
s = cross_val_score(GaussianNB(), X, y, cv=KFold(10, shuffle=True), scoring=make_scorer(accuracy_score))
print ("X shape", X.shape)
print ("accuracy %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))

In [ ]:
X = PCA(n_components=3).fit_transform(X)
print ("shape after PCA ", X.shape)
s = cross_val_score(GaussianNB(), X, y, cv=KFold(10, shuffle=True), scoring=make_scorer(accuracy_score))
print ("accuracy %.3f (+/- %.5f)"%(np.mean(s), np.std(s)))